In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os

# استعراض جميع المجلدات داخل /kaggle/input
print(os.listdir("/kaggle/input"))


['meta-kaggle']


In [2]:
print(os.listdir("/kaggle/input/meta-kaggle"))

['KernelTags.csv', 'ModelVariations.csv', 'KernelVersionCompetitionSources.csv', 'Datasets.csv', 'KernelVersionKernelSources.csv', 'KernelVotes.csv', 'Submissions.csv', 'KernelLanguages.csv', 'Users.csv', 'ForumMessageVotes.csv', 'Competitions.csv', 'DatasetTaskSubmissions.csv', 'UserAchievements.csv', 'UserOrganizations.csv', 'Teams.csv', 'UserFollowers.csv', 'CompetitionTags.csv', 'Kernels.csv', 'Organizations.csv', 'Datasources.csv', 'ModelVersions.csv', 'ForumTopics.csv', 'DatasetVersions.csv', 'ModelVotes.csv', 'DatasetVotes.csv', 'TeamMemberships.csv', 'Forums.csv', 'KernelVersions.csv', 'ModelVariationVersions.csv', 'ForumMessages.csv', 'KernelVersionDatasetSources.csv', 'Episodes.csv', 'EpisodeAgents.csv', 'KernelAcceleratorTypes.csv', 'KernelVersionModelSources.csv', 'ForumMessageReactions.csv', 'Tags.csv', 'DatasetTasks.csv', 'Models.csv', 'DatasetTags.csv', 'ModelTags.csv']


In [8]:
import pandas as pd
import time, psutil

def memory_usage_mb():
    process = psutil.Process()
    return process.memory_info().rss / (1024 ** 2)

start = time.time()

files_to_read = [
    "Kernels.csv",
    "KernelVersions.csv",
    "Datasets.csv"
]

dfs = []
for f in files_to_read:
    file_path = f"/kaggle/input/meta-kaggle/{f}"
    chunks = pd.read_csv(file_path, chunksize=100000)
    df = pd.concat(chunks)
    dfs.append(df)

df_pandas = pd.concat(dfs, axis=0)
print(f"Pandas read done in {time.time() - start:.2f}s")
print(f"Memory used: {memory_usage_mb():.2f} MB")


Pandas read done in 141.93s
Memory used: 21764.79 MB


In [12]:
import dask.dataframe as dd
import time
import warnings
warnings.filterwarnings("ignore")

start = time.time()
paths = [f"/kaggle/input/meta-kaggle/{f}" for f in files_to_read]
df_dask = dd.read_csv(paths, assume_missing=True)
print("Dask preview:")
print(df_dask.head())

print(f"Dask read time: {time.time() - start:.2f}s")
print(f"Memory used: {memory_usage_mb():.2f} MB")


Dask preview:
        Id  AuthorUserId  CurrentKernelVersionId  ForkParentKernelVersionId  \
0    297.0         368.0                  1760.0                     1717.0   
1    314.0         368.0                  1810.0                        NaN   
2    316.0         368.0                  1809.0                        NaN   
3    599.0         368.0                  2802.0                     1275.0   
4  15297.0         368.0                 55821.0                        NaN   

   ForumTopicId  FirstKernelVersionId         CreationDate EvaluationDate  \
0           NaN                1750.0  04/16/2015 17:42:31     04/16/2015   
1           NaN                1783.0  04/17/2015 23:33:44     04/18/2015   
2       17103.0                1806.0  04/18/2015 00:14:28     04/18/2015   
3       51451.0                2523.0  04/24/2015 16:55:05     04/25/2015   
4           NaN               55817.0  08/26/2015 01:53:50     08/26/2015   

  MadePublicDate  IsProjectLanguageTemplate  \
0

In [6]:
import os, gzip, shutil, time, psutil, pandas as pd

folder = "/kaggle/input/meta-kaggle"


files_to_compress = [
    "Kernels.csv",         
    "KernelVersions.csv",   
    "Datasets.csv",         
]

# دالة حساب استهلاك الذاكرة
def memory_usage_mb():
    process = psutil.Process()
    return process.memory_info().rss / (1024 ** 2)

# ضغط الملفات وحفظها في working directory
for f in files_to_compress:
    src = f"{folder}/{f}"
    dst = f"/kaggle/working/{f}.gz"

    print(f" Compressing {f} ...")
    start = time.time()
    with open(src, "rb") as fin, gzip.open(dst, "wb") as fout:
        shutil.copyfileobj(fin, fout)
    print(f" Done: {dst}  |  Time: {time.time() - start:.2f}s\n")

print("All selected files compressed successfully!\n")
print("Compressed files:", os.listdir("/kaggle/working"))


 Compressing Kernels.csv ...
 Done: /kaggle/working/Kernels.csv.gz  |  Time: 90.84s

 Compressing KernelVersions.csv ...
 Done: /kaggle/working/KernelVersions.csv.gz  |  Time: 706.76s

 Compressing Datasets.csv ...
 Done: /kaggle/working/Datasets.csv.gz  |  Time: 33.95s

All selected files compressed successfully!

Compressed files: ['Datasets.csv.gz', 'KernelVersions.csv.gz', 'Kernels.csv.gz', '.virtual_documents']


In [ ]:
start_time = time.time()
start_mem = memory_usage_mb()

dfs = []
for f in files_to_compress:
    gz_path = f"/kaggle/working/{f}.gz"
    print(f" Reading compressed file: {gz_path}")
    df = pd.read_csv(gz_path, compression='gzip')
    dfs.append(df)

df_all = pd.concat(dfs, axis=0)

read_time = time.time() - start_time
used_mem = memory_usage_mb() - start_mem

print("\n Compression Method Finished Successfully!")
print(f"Time taken: {read_time:.2f} seconds")
print(f" Memory used: {used_mem:.2f} MB")
print(f" Total rows read: {len(df_all):,}")


📘 Reading compressed file: /kaggle/working/Kernels.csv.gz
📘 Reading compressed file: /kaggle/working/KernelVersions.csv.gz


/tmp/ipykernel_37/1649400246.py:8: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(gz_path, compression='gzip')


📘 Reading compressed file: /kaggle/working/Datasets.csv.gz

 Compression Method Finished Successfully!
Time taken: 134.73 seconds
 Memory used: 8860.53 MB
 Total rows read: 18,265,835


In [ ]:
تحليل النتائج
1. من حيث السرعة

Dask :هي الأسرع في مرحلة التحميل (لأنها لا تقرأ كل البيانات دفعة واحدة)

2. من حيث استهلاك الذاكرة

Pandas :حمّلت كل الملفات في الذاكرة  GB 21.7 
Dask : حافظ على نفس المستوى تقريبًا لأنه في النهاية يجب أن يحمل البيانات للمعالجة.

طريقة الضغط (compression) خفّضت الذاكرة المستعملة إلى أقل من النصف (≈8.8 GB) .
خلاصة القول في حالة الملفات الضخمة  نجد اسرع طرقة للقراءة هي dask read method و اقلهم من ناحية الذاكرة هي compression read method